# 策略选股

1. 选股原理

一般来说，具备高送转预期的个股，都具有总市值低、每股公积金高、每股收益大，流通股本少的特点。当然，也还有其它的因素，比如当前股价、经营收益变动情况、以及以往分红送股习惯等等。

这里我们暂时只考虑每股公积金、每股收益、流通股本和总市值四个因素，将公积金大于等于5元，每股收益大于等于5毛，流通股本在3亿以下，总市值在100亿以内作为高送转预期目标.

# 1 数据清洗

 1.1 当前股价,如果停牌则设置当前价格为上一个交易日股价

 1.2 分别选取流通股本,总股本,每股公积金,每股收益
 
 1.3 选取股票代码,名称,当前价格,总市值,流通市值

In [2]:
import tushare as ts

In [3]:
# 基本面数据
basic = ts.get_stock_basics()

# 行情和市值数据
hq = ts.get_today_all()

[Getting data:]############################################################

In [4]:
#当前股价,如果停牌则设置当前价格为上一个交易日股价
hq['trade'] = hq.apply(lambda x:x.settlement if x.trade==0 else x.trade, axis=1)

#分别选取流通股本,总股本,每股公积金,每股收益
basedata = basic[['outstanding', 'totals', 'reservedPerShare', 'esp']]

#选取股票代码,名称,当前价格,总市值,流通市值
hqdata = hq[['code', 'name', 'trade', 'mktcap', 'nmc']]

#设置行情数据code为index列
hqdata = hqdata.set_index('code')

#合并两个数据表
data = basedata.merge(hqdata, left_index=True, right_index=True)

In [9]:
data.head()

,outstanding,totals,reservedPerShare,esp,name,trade,mktcap,nmc
code,,,,,,,,
000001,171.70,171.70,3.29,0.730,平安银行,9.84,1689.568478,1689.552282
000002,97.15,110.39,0.78,0.826,万 科Ａ,23.32,2574.330247,2265.577654
000004,0.83,0.84,0.01,0.024,国农科技,15.84,13.301907,13.149288
000005,9.51,10.59,0.65,0.008,世纪星源,2.96,31.332691,28.156185
000006,13.48,13.50,0.36,0.308,深振业Ａ,5.12,69.119746,69.033368


In [6]:
# 将总市值和流通市值换成亿元单位
data['mktcap'] = data['mktcap'] / 10000
data['nmc'] = data['nmc'] / 10000

#每股公积金>=5
res = data.reservedPerShare >= 5
#流通股本<=3亿
out = data.outstanding <= 30000
#每股收益>=5毛
eps = data.esp >= 0.5
#总市值<100亿
mktcap = data.mktcap <= 100

selected = data[res & out & eps & mktcap]

In [7]:
# name股票名称、trad收盘价格、reservedPerShare每股公积金、outstanding流通股本、每股收益（应该为eps，之前发布笔误）、mktcap总市值和nmc流通市值
selected.head()

,outstanding,totals,reservedPerShare,esp,name,trade,mktcap,nmc
code,,,,,,,,
000525,5.07,5.81,5.07,0.657,红 太 阳,14.15,82.179362,71.738108
002712,1.81,3.42,6.23,0.509,思美传媒,16.12,55.111671,29.218201
002718,0.26,0.88,5.13,0.603,友邦吊顶,26.61,23.318845,6.927085
002757,0.97,1.31,7.57,0.670,南兴装备,29.14,38.297551,28.409701
002798,0.55,2.27,10.57,0.730,帝欧家居,29.89,67.766832,16.311085


# 金叉判断

In [ ]:
# 金叉判断
def jincha(his):    
    def zs5(his):
        #站上5日线
        ma_n = pd.rolling_mean(his, 5)
        temp = his - ma_n
        #temp_s包含了前一天站上五日线得股票代码
        temp_s = list(temp[temp>0].iloc[-1,:].dropna().index)
        return temp_s    

    def zs10(his):
        #站上10日线
        ma_n = pd.rolling_mean(his, 10)
        temp = his - ma_n
        temp_s = list(temp[temp>0].iloc[-1,:].dropna().index)
        return temp_s    

    def jc(his):
        #金叉突破
        mas = pd.rolling_mean(his,5)
        mal = pd.rolling_mean(his, 10)
        temp = mas - mal
        #temp_jc昨天大于0股票代码
        #temp_r前天大于0股票代码
        temp_jc = list(temp[temp>0].iloc[-1,:].dropna().index)
        temp_r = list(temp[temp>0].iloc[-2,:].dropna().index)
        temp = []
        for stock in temp_jc:
            if stock not in temp_r:
                temp.append(stock)
        return temp    

    #求三种条件下的股票代码交集
    con1 = zs5(his)
    con2 = zs10(his)
    con3 = jc(his)
    tar_list=[con1,con2,con3]
    tarstock = tar_list[0]
    for i in tar_list:
        tarstock = list(set(tarstock).intersection(set(i)))
    return tarstock

In [ ]:
d = jincha()

In [ ]:
#过滤次新股、是否涨跌停、是否停牌等条件

def filcon(context,bar_dict,tar_list):

    def zdt_trade(stock, context, bar_dict):

        yesterday = history(2,'1d', 'close')[stock].values[-1]
        zt = round(1.10 * yesterday,2)
        dt = round(0.99 * yesterday,2)
        #last最后交易价

        return dt < bar_dict[stock].last < zt

    filstock = []

    for stock in tar_list:

        con1 = ipo_days(stock,context.now) > 60
        con2 = bar_dict[stock].is_trading
        con3 = zdt_trade(stock,context,bar_dict)

        if con1 & con2 & con3:
                filstock.append(stock)

    return filstock

# 按平均持仓市值调仓

In [ ]:
# 按平均持仓市值调仓
def for_balance(context, bar_dict):
    #mvalues = context.portfolio.market_value
    #avalues = context.portfolio.portfolio_value
    #per = mvalues / avalues
    hlist = []
    for stock in context.portfolio.positions:
        #获取股票及对应持仓市值 
        hlist.append([stock,bar_dict[stock].last * context.portfolio.positions[stock].quantity])
    
    if hlist:
        #按持仓市值由大到小排序
        hlist = sorted(hlist,key=lambda x:x[1], reverse=True)
        temp = 0
        for li in hlist:
            #计算持仓总市值
            temp += li[1]
        for li in hlist:
            #平均各股持仓市值
            if bar_dict[li[0]].is_trading:
                order_target_value(li[0], temp/len(hlist))
    return